# 3D Image Classification from CT Scans

In [30]:
# %pip install nibabel

In [31]:
# %pip install scipy

In [32]:
import tensorflow as tf 
import os 
import zipfile 
import numpy as np

## Downloading the MosMedData: Chest CT Scans with Covid-19 Related Findings
In this example, we use a subset of the
[MosMedData: Chest CT Scans with COVID-19 Related Findings](https://www.medrxiv.org/content/10.1101/2020.05.20.20100362v1).
This dataset consists of lung CT scans with COVID-19 related findings, as well as without such findings.

We will be using the associated radiological findings of the CT scans as labels to build
a classifier to predict presence of viral pneumonia.
Hence, the task is a binary classification problem.

In [33]:
# Download normal CT scans.
# url = "https://github.com/hasibzunair/3D-image-classification-tutorial/releases/download/v0.2/CT-0.zip"
# filename = os.path.join(os.getcwd(), "CT-0.zip")
# tf.keras.utils.get_file(cache_dir=filename, origin=url)

In [34]:
# Download abnormal CT scans.
# url = "https://github.com/hasibzunair/3D-image-classification-tutorial/releases/download/v0.2/CT-23.zip"
# filename = os.path.join(os.getcwd(), "CT-23.zip")
# tf.keras.utils.get_file(cache_dir=filename, origin=url)

In [35]:
# with zipfile.ZipFile('CT-0.zip','r') as z_fp: 
#     z_fp.extractall('./MosMedData')

# with zipfile.ZipFile('CT-23.zip','r') as z_fp: 
#     z_fp.extractall('./MosMedData')

## Loading Data and Prepocessing 
The files are provided in Nifti format with the extension .nii. To read the
scans, we use the `nibabel` package.
CT scans store raw voxel intensity in Hounsfield units (HU). They range from -1024 to above 2000 in this dataset.
Above 400 are bones with different radiointensity, so this is used as a higher bound. A threshold
between -1000 and 400 is commonly used to normalize CT scans.

To process the data, we do the following:

* We first rotate the volumes by 90 degrees, so the orientation is fixed
* We scale the HU values to be between 0 and 1.
* We resize width, height and depth.

Here we define several helper functions to process the data. These functions
will be used when building training and validation datasets.

In [44]:
import nibabel as nib 
from scipy import ndimage
import random

In [37]:
def read_nifti_file(filepath): 
    "Read and Load volume"
    scan = nib.load(filepath)
    scan = scan.get_fdata()
    return scan 

In [38]:
def normalize(volume): 
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max

    volume = (volume - min) / (max - min)
    volume = volume.astype('float32')
    return volume

In [39]:
def resize_volume(img):  
    "Resize across z-axis"
    # Set the desired dimensions of the image
    desired_depth = 64
    desired_width = 128
    desired_height = 128

    # Get the current dimensions of the image
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]

    # Calculate the scaling factors for each dimension
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height

    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height

    # Rotate  the image by 90 deg
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize the image using the calculated scaling factors
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

In [40]:
def process_scan(path): 
    "Read and Resize Volume"
    volume = read_nifti_file(path)
    volume = resize_volume(volume)
    volume = resize_volume(volume)
    return volume

Let's read the paths of the CT scans from the class directories.

In [41]:
normal_scan_paths = [
    os.path.join(os.getcwd(), "MosMedData/CT-0", x)
    for x in os.listdir("MosMedData/CT-0")
]


In [42]:
abnormal_scan_paths = [
    os.path.join(os.getcwd(), "MosMedData/CT-23", x)
    for x in os.listdir("MosMedData/CT-23")
]
print("CT scans with normal lung tissue: " + str(len(normal_scan_paths)))
print("CT scans with abnormal lung tissue: " + str(len(abnormal_scan_paths)))

CT scans with normal lung tissue: 100
CT scans with abnormal lung tissue: 100


## Split Data 
Read the scans from the class directories and assign labels. Downsample the scans to have
shape of 128x128x64. Rescale the raw HU values to the range 0 to 1.
Lastly, split the dataset into train and validation subsets.

In [43]:
normal_scans = np.array([process_scan(path) for path in normal_scan_paths])
abnormal_scans = np.array([process_scan(path) for path in abnormal_scan_paths])

normal_labels = np.array([0 for _ in range(len(normal_scans))])
abnormal_labels = np.array([1 for _ in range(len(abnormal_scans))])

split_ratio = 0.7

split_index_normal = int(len(normal_scans) * split_ratio)
split_index_abnormal = int(len(abnormal_scans) * split_ratio)

X_train = np.concatenate((normal_scans[:split_index_normal],abnormal_scans[:split_index_abnormal]), axis=0)
y_train = np.concatenate((normal_labels[:split_index_normal], abnormal_labels[:split_index_abnormal]), axis=0)
x_val = np.concatenate((normal_scans[split_index_normal:], abnormal_scans[split_index_abnormal:]), axis=0)
y_val = np.concatenate((normal_labels[:split_index_normal:], abnormal_labels[split_index_abnormal:]), axis=0)

print(
    "Number of samples in train and validation are %d and %d."
    % (X_train.shape[0], x_val.shape[0])
)

Number of samples in train and validation are 140 and 60.


## Data Augmentation

The CT scans also augmented by rotating at random angles during training. Since
the data is stored in rank-3 tensors of shape `(samples, height, width, depth)`,
we add a dimension of size 1 at axis 4 to be able to perform 3D convolutions on
the data. The new shape is thus `(samples, height, width, depth, 1)`. There are
different kinds of preprocessing and augmentation techniques out there,
this example shows a few simple ones to get started.

In [ ]:
def rotate(volume): 
    "Rotate the volume by few degrees"
    
    def scipy_rotate(volume): 
        angles = [-20, -10, -5, 5, 10, 20]
        angle = random.choice(angles) # pick angle at random

        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume
    
    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)


